In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import json
import cv2

import seaborn as sn
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

import numpy as np

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# mount google drive
# file authorization

# from google.colab import drive

# drive.mount('/content/drive')

In [ ]:
# # open file from 
# try:
#   zipLeafImages = ZipFile('/content/drive/My Drive/Data/cassavaleafdiseaseclassification.zip')
#   zipLeafImages.extractall()
# except:
#   print('file not found !!!')
#   exit()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
BASE_DIR = "../input/cassava-leaf-disease-classification/"
# BASE_DIR = "/content/drive/MyDrive/Data/cassava-leaf-disease-classification/"

In [ ]:
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    map_classes = {int(k) : v for k, v in map_classes.items()}

print(json.dumps(map_classes, indent=4))

In [ ]:
inputs_files = os.listdir(os.path.join(BASE_DIR, "train_images"))
print(f"Number of input images : {len(inputs_files)}")

Get the images shape, shape should be same for all images

In [ ]:
image_shape = {}
for image_name in os.listdir(os.path.join(BASE_DIR, "train_images"))[:5]:
    images = cv2.imread(os.path.join(BASE_DIR, "train_images", image_name))
    image_shape[images.shape] = image_shape.get(images.shape, 0) + 1

print(image_shape)

Merging images with labels

In [ ]:
df_train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))
df_train['class_name'] = df_train['label'].map(map_classes)
df_train = df_train.sample(n=1000)
df_train.shape

In [ ]:
plt.figure(figsize = (12, 8))
sn.countplot(y = "class_name", data = df_train)

Labeling data in different way

In [ ]:
def f(row):
    if row['label'] == 0:
        val = 1
    else:
        val = 0
    return val

In [ ]:
def e(row):
    if row['label'] == 3:
        val = 1
    else:
        val = 0
    return val

In [ ]:
def d(row):
    if row['label'] == 1:
        val = 1
    else:
        val = 0
    return val

In [ ]:
def c(row):
    if row['label'] == 4:
        val = 1
    else:
        val = 0
    return val

In [ ]:
def b(row):
    if row['label'] == 2:
        val = 1
    else:
        val = 0
    return val

In [ ]:
df_train_with_label = df_train
df_train_with_label['class_name'] = df_train['label'].map(map_classes)
df_train_with_label['cbb'] = df_train_with_label.apply(f, axis=1)
df_train_with_label['cmd'] = df_train_with_label.apply(e, axis=1)
df_train_with_label['cbsd'] = df_train_with_label.apply(d, axis=1)
df_train_with_label['cgm'] = df_train_with_label.apply(b, axis=1)
df_train_with_label['healthy'] = df_train_with_label.apply(c, axis=1)
df_train_with_label.reset_index(drop=True, inplace=True)
df_train_with_label

In [ ]:
y = np.array(df_train_with_label.drop(['image_id','label', 'class_name'],axis=1))

In [ ]:
y.shape

In [ ]:
# df_train_with_label

In [ ]:
plt.figure(figsize=(10,10))
folder= BASE_DIR + '/train_images'
for images in os.listdir(os.path.join(BASE_DIR, "train_images"))[:30]:
    ax = plt.subplot(3,3, random.randint(1,9))
    img = cv2.imread(os.path.join(folder,images))
    plt.imshow(img)
    plt.axis("off")

Lets see image for each deases

# Visual Image Selector

In [ ]:
def print_random_images(label):
    plt.figure(figsize=(10,10))
    folder= BASE_DIR + '/train_images'
    cbb_images = df_train[df_train.label == label]
    for images in cbb_images.image_id.head(29):
        ax = plt.subplot(3,3, random.randint(1,9))
        img = cv2.imread(os.path.join(folder,images))
        plt.imshow(img)
        plt.axis("off")

# Data needs to be labeled before to identify is our module did the job or not

In [ ]:
df_train.columns

In [ ]:
df_train.reset_index(drop=True, inplace=True)
df_train

In [ ]:
train_image = []
for i in tqdm(range(df_train.shape[0])):
    img = image.load_img(BASE_DIR + 'train_images/'+df_train['image_id'][i],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)

X = np.array(train_image)

In [ ]:
def transfer_mages(df_train_pool):
  train_image = []
  for i in tqdm(range(df_train_pool.shape[0])):
    img = image.load_img(BASE_DIR + 'train_images/'+df_train_pool['image_id'][i],target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
    
  X = np.array(train_image)
  return X

In [ ]:
X.shape

In [ ]:
plt.imshow(X[2])

#     "0": "Cassava Bacterial Blight (CBB)",

In [ ]:
print_random_images(0)

#     "1": "Cassava Brown Streak Disease (CBSD)"

In [ ]:
print_random_images(1)

# "2": "Cassava Green Mottle (CGM)"

In [ ]:
print_random_images(2)

#    "3": "Cassava Mosaic Disease (CMD)"


In [ ]:
print_random_images(3)

#    "4": "Healthy"

In [ ]:
print_random_images(4)

# Make a model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
# model = Sequential()
# model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(400,400,3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(5, activation='sigmoid'))

In [ ]:
image_size = (400, 400)

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    x = data_augmentation(inputs)

    # Entry block
    x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(x)
    
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=5)
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
X_test.shape, y_test.shape

In [ ]:
X_train.shape, y_train.shape

In [ ]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=16)

# Test

In [ ]:
img = image.load_img(BASE_DIR+'test_images/2216849948.jpg',target_size=(400,400,3))
img = image.img_to_array(img)
img = img/255

In [ ]:
classes = np.array(df_train.columns[3:])
proba = model.predict(img.reshape(1,400,400,3))
top_3 = np.argsort(proba[0])
for i in range(5):
    print("{}".format(classes[top_3[i]])+" ({:.3})".format(proba[0][top_3[i]]))
plt.imshow(img)

In [ ]:
output = dict(zip(classes[top_3],proba.ravel()))
output_value = max(output, key=output.get)
output_value

In [ ]:
val = 4
def get_label_value(label):
    if label == 'cbb':
        val = 0
    elif label == 'cbsd':
        val = 1
    elif label == 'cgm':
        val = 2
    elif label == 'cmd':
        val = 3
    else:
        val = 4
    return val

In [ ]:
test_df = pd.DataFrame({'image_id': ['2216849948.jpg']})
test_df['label'] = get_label_value(output_value)
test_df

In [ ]:
test_df.to_csv('submission.csv', encoding='utf-8')